In [1]:
import numpy as np
import pandas as pd
from liwc import Liwc
from matplotlib import pyplot as plt
import seaborn as sns
import json
from tqdm import tqdm
import scipy
import glob
import re
import yaml
import warnings
import nltk
import spacy
warnings.filterwarnings("ignore")
tqdm.pandas()

import scipy.stats as stats

filepaths = yaml.safe_load(open("../../../config/filepaths.yaml"))

import sys
sys.path.append(filepaths['utils'])
from data import *
from liwc_utils import *
from analisys import *

/home/luiznery/anaconda3/envs/fake/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rcParams['axes.labelsize']  = 14 # fontes no tamanho usado no template
plt.rcParams['axes.titlesize']  = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['font.family'] = 'serif'
# plt.rcParams['text.usetex'] = True
plt.rcParams['lines.linewidth'] = 1.5

plt.style.use('tableau-colorblind10') # use um estilo colorblind!
plt.ion()

---

In [9]:
DATASET = 'twitter' # 'coraal-buckeye' | 'twitter' | 'youtube' | 'interview'
BLACK = 'aa'
WHITE = 'wh'

data = load_dataset(DATASET)
data.head(1)

,text,perspective_score,flair_score,textblob_score,vader_score,detoxify_original_score,detoxify_unbiased_score,detoxify_multilingual_score,has_swearing,file,group
0,@user wat r u doin boy,0.037538,0.0,0.0,0.0,0.048113,0.014581,0.146005,False,aa_112.csv,aa


In [4]:
df = data.copy()
df['text'] = df['text'].apply(lambda x: re.sub("[^\w\d'\s]+",'',x))
df['len'] = df.text.str.split(' ').str.len()
df['words'] = df.text.str.split(' ')

---

In [5]:
nlp = spacy.load("en_core_web_sm")


In [6]:
df['POS'] = df.text.progress_apply(lambda x: [(tag.text,tag.pos_) for tag in nlp(x)])

100%|██████████| 500/500 [00:03<00:00, 152.73it/s]


---

In [7]:
def check_ocurence_in_list(sequence,pos_list):
    """
    count the number of times a sequence appears in a list
    """
    i = 0
    while i < len(pos_list)-len(sequence)+1:
        #            if (sequence[j][0] == None and sequence[j][1] == None) or (sequence[j][0] != None and sequence[j][1] != None): raise Exception("Bad input sequence")#
    
        is_eq = True
        for j in range(len(sequence)):
            if sequence[j][0] == None and sequence[j][1] != pos_list[i+j][1]:
                is_eq = False
                break
            elif sequence[j][0] != None and sequence[j][0] != pos_list[i+j][0]:
                is_eq = False
                break
            
        if is_eq:
            return 1

        i += 1

    return 0

def count_sequence(df,sequence,group):
    return df[df.group.str.contains(group)].POS.apply(lambda x: check_ocurence_in_list(sequence,x)).sum()

def contain_sequence_idx(df,sequence):
    return df.POS.apply(lambda x: check_ocurence_in_list(sequence,x)) > 0

def group_total(df,group):
    return len(df[df.group.str.contains(group)])

def calc_sequence_freq(df,sequence,group):    
    return count_sequence(df,sequence,group) / group_total(df,group)


### O - b/be - V -> PRON - be - VERB

In [10]:
seq = [(None,'PRON'), ('be',None), (None,'VERB')]
print(calc_sequence_freq(df,seq,BLACK), count_sequence(df,seq,BLACK), group_total(df,BLACK))
print(calc_sequence_freq(df,seq,WHITE), count_sequence(df,seq,WHITE), group_total(df,WHITE))

0.004 1 250
0.0 0 250



### gone - V -> gone - VERB

In [11]:
seq = [('gone',None), (None,'VERB')]
print(calc_sequence_freq(df,seq,BLACK), count_sequence(df,seq,BLACK), group_total(df,BLACK))
print(calc_sequence_freq(df,seq,WHITE), count_sequence(df,seq,WHITE), group_total(df,WHITE))

0.004 1 250
0.0 0 250



### done - V -> done - VERB

In [12]:
seq = [('done',None), (None,'VERB')]
print(calc_sequence_freq(df,seq,BLACK), count_sequence(df,seq,BLACK), group_total(df,BLACK))
print(calc_sequence_freq(df,seq,WHITE), count_sequence(df,seq,WHITE), group_total(df,WHITE))

0.004 1 250
0.004 1 250
